In [2]:
!python -V

Python 3.11.9


In [3]:
import pandas as pd

In [4]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [7]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='file:///c:/Users/arun_prasath/workspace/courses/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1719996657207, experiment_id='1', last_update_time=1719996657207, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [10]:
len(df_train), len(df_val)

(54373, 51497)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


5.995083318320955

In [15]:
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "../data/green_tripdata_2024-01.parquet")
    mlflow.log_param("valid-data-path", "../data/green_tripdata_2024-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="../models/lin_reg.bin", artifact_path="models_pickle")

c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [17]:
import xgboost as xgb

In [18]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [20]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [21]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:14:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.93917                           
[1]	validation-rmse:7.09039                           
[2]	validation-rmse:6.48602                           
[3]	validation-rmse:6.06816                           
[4]	validation-rmse:5.78482                           
[5]	validation-rmse:5.59209                           
[6]	validation-rmse:5.46501                           
[7]	validation-rmse:5.38168                           
[8]	validation-rmse:5.32558                           
[9]	validation-rmse:5.29144                           
[10]	validation-rmse:5.26402                          
[11]	validation-rmse:5.24835                          
[12]	validation-rmse:5.23623                          
[13]	validation-rmse:5.22766                          
[14]	validation-rmse:5.22075                          
[15]	validation-rmse:5.21668                          
[16]	validation-rmse:5.21265                          
[17]	validation-rmse:5.20864                          
[18]	valid

c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:14:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.47513                                                    
[1]	validation-rmse:6.51375                                                    
[2]	validation-rmse:5.97590                                                    
[3]	validation-rmse:5.67954                                                    
[4]	validation-rmse:5.52285                                                    
[5]	validation-rmse:5.42893                                                    
[6]	validation-rmse:5.37640                                                    
[7]	validation-rmse:5.34230                                                    
[8]	validation-rmse:5.31808                                                    
[9]	validation-rmse:5.30281                                                    
[10]	validation-rmse:5.29217                                                   
[11]	validation-rmse:5.28326                                                   
[12]	validation-rmse:5.27605            

...

c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:41:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.70736                                                     
[1]	validation-rmse:8.34048                                                     
[2]	validation-rmse:8.00723                                                     
[3]	validation-rmse:7.70655                                                     
[4]	validation-rmse:7.43525                                                     
[5]	validation-rmse:7.18961                                                     
[6]	validation-rmse:6.96901                                                     
[7]	validation-rmse:6.77140                                                     
[8]	validation-rmse:6.59393                                                     
[9]	validation-rmse:6.43465                                                     
[10]	validation-rmse:6.29366                                                    
[11]	validation-rmse:6.16838                                                    
[12]	validation-rmse:6.05611

c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:42:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.60850                                                     
[1]	validation-rmse:8.16780                                                     
[2]	validation-rmse:7.77431                                                     
[3]	validation-rmse:7.44227                                                     
[4]	validation-rmse:7.14324                                                     
[5]	validation-rmse:6.88661                                                     
[6]	validation-rmse:6.65472                                                     
[7]	validation-rmse:6.46313                                                     
[8]	validation-rmse:6.29470                                                     
[9]	validation-rmse:6.14918                                                     
[10]	validation-rmse:6.02423                                                    
[11]	validation-rmse:5.92184                                                    
[12]	validation-rmse:5.83456

c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [22]:
mlflow.xgboost.autolog(disable=True)

In [23]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("../models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:43:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.55121
[1]	validation-rmse:8.06400
[2]	validation-rmse:7.64347
[3]	validation-rmse:7.28077
[4]	validation-rmse:6.96868
[5]	validation-rmse:6.70403
[6]	validation-rmse:6.47713
[7]	validation-rmse:6.28420
[8]	validation-rmse:6.12122
[9]	validation-rmse:5.98447
[10]	validation-rmse:5.87103
[11]	validation-rmse:5.77302
[12]	validation-rmse:5.68988
[13]	validation-rmse:5.62238
[14]	validation-rmse:5.56459
[15]	validation-rmse:5.51506
[16]	validation-rmse:5.47349
[17]	validation-rmse:5.43747
[18]	validation-rmse:5.40878
[19]	validation-rmse:5.38302
[20]	validation-rmse:5.36147
[21]	validation-rmse:5.34029
[22]	validation-rmse:5.32216
[23]	validation-rmse:5.30792
[24]	validation-rmse:5.29485
[25]	validation-rmse:5.28297
[26]	validation-rmse:5.27297
[27]	validation-rmse:5.26478
[28]	validation-rmse:5.25749
[29]	validation-rmse:5.25086
[30]	validation-rmse:5.24386
[31]	validation-rmse:5.23724
[32]	validation-rmse:5.23377
[33]	validation-rmse:5.23046
[34]	validation-rmse:5.2

c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:44:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [26]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "../data/green_tripdata_2023-01.parquet")
        mlflow.log_param("valid-data-path", "../data/green_tripdata_2023-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2024/07/03 18:45:29 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/07/03 18:45:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils."
2024/07/03 18:45:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\arun_prasath\AppData\Local\anaconda3\e

AssertionError: c:\Users\arun_prasath\AppData\Local\anaconda3\envs\mlops\Lib\distutils\core.py